In [1]:
import pandas as pd
import numpy as np
import os
from ckiptagger import construct_dictionary, WS, POS, NER
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.metrics import f1_score

In [59]:
# 匯入label好的資料集
df = pd.read_csv('labeled_data/2022-01-22.csv', encoding='utf-8', header = None)
df.columns=['label', 'text']
df.drop(df.index[(df.label != 0) & (df.label != 1) & (df.label != 2)], inplace = True)
df.reset_index(drop=True, inplace = True)

In [146]:
df

,label,text,cut_text
0,2.0,先知 尊敬你 酸民哭哭,先知 尊敬 你 酸民 哭哭
1,0.0,台星現在還是虧錢吧 台哥願意併 有談好條件價碼觀望,台星 現在 還是 虧錢 吧 台哥 願意 併 有 談好 條件 價碼 觀望
2,0.0,明天有開盤嗎？總加速師，讓各位韭菜回家吃自己黨宣布經濟成長騰飛8.1%可是同時政府破產又公務...,明天 有 開盤 嗎 ？ 總加速師 ， 讓 各位 韭菜 回家吃自己 黨 宣布 經濟 成長 騰飛...
3,1.0,賺錢特地回來感謝鬆哥 這是很準我是打算買個幾張放著看戲,賺錢 特地 回來 感謝 鬆哥 這 是 很準 我 是 打算 買個幾張放著 看戲
4,1.0,其實這題材還行 畢竟他算是 電動車跟元宇宙都有涉獵到 但什麼時候可以廣泛應用 還說不準 怕...,其實 這 題材 還 行 畢竟 他 算是 電動車 跟 元 宇宙 都 有 涉獵到 但...
...,...,...,...
4719,0.0,※ 引述《thumbd000 (藤原千花)》之銘言： : 從國際股市來看 : 美股爆殺到現在...,※ 引述 《 thumbd 000 ( 藤原千花 ) 》 之 銘言 ： : 從 ...
4720,2.0,...,...
4721,2.0,不錯阿，說得很好,不錯 阿 ， 說 得 很好
4722,2.0,大陸之前打通膨是為了之後的寬鬆政策吧?熱錢都流到大陸也不用怕通膨美國升息縮表都不能把錢從大陸吸走?,大陸 之前 打通膨 是 為 了 之後 的 寬鬆 政策 吧 ? 熱錢 都 流到 大陸 也 不用...


In [85]:
# 匯入stopwords
def import_stopwords():
    # 匯入stopwords
    with open("stopwords.csv",'r',encoding='utf-8') as f:
        stopword_file = pd.read_csv(f, header=None)
    stopwords_list = stopword_file.values.tolist()
    stopwords = set()
    for s in stopwords_list:
        for word in s:
            if pd.isnull(word) == False:
                if type(word) == float:
                    word = str(int(word))
                stopwords.add(word)
    return list(stopwords)
stopwords = import_stopwords()

In [61]:
# 使用ckip斷詞
def ckip(sentence):
    # 匯入自定斷詞
    with open('customized.csv','r',encoding='utf-8') as f:
        customized_file = pd.read_csv(f, header=None)
    customized_list = customized_file.values.tolist()
    
    # 整理customized格式
    customized = set()
    for c in customized_list:
        for word in c:
            if pd.isnull(word) == False:
                if type(word) == float:
                    word = str(int(word))
                customized.add(word)
    customized.update(stopwords)
    customized = list(customized)

    # 給斷詞權重   
    word_to_weight = {}
    for c in customized:
        word_to_weight.update({c : 1})
    
    # 利用ckip斷詞
    dictionary = construct_dictionary( word_to_weight )
    ws = WS("./data") 
    ws_results = ws(sentence, 
                    # recommend_dictionary = dictionary,
                    coerce_dictionary = dictionary)
    return ws_results
ws_results = ckip(df.text)

# 將斷詞完成的資料存成dataframe
cut_text = [" ".join(i) for i in ws_results]
df['cut_text'] = pd.DataFrame(cut_text)

/opt/anaconda3/lib/python3.9/site-packages/ckiptagger/model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)


In [66]:
# 分別存入 X 跟 y
X = df[['cut_text']]
y = df.label.astype(int)

In [67]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [116]:
vect = CountVectorizer()
term_matrix = pd.DataFrame(vect.fit_transform(X_train.cut_text).toarray(), 
                           columns = vect.get_feature_names_out())
term_matrix.shape

(3543, 7391)

In [118]:
vect = CountVectorizer(stop_words = frozenset(stopwords))
term_matrix = pd.DataFrame(vect.fit_transform(X_train.cut_text).toarray(), 
                           columns = vect.get_feature_names_out())
term_matrix.shape

/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['c螺', 'ettoday', '其他', '助詞', '動詞', '名詞', '單位量詞冠詞', '大家好我是will', '感嘆情緒', '成語俚語', '時間', '柯p', '稱呼人名', '符號', '處份', '財經m平方', '連接詞'] not in stop_words.
  warnings.warn(


(3543, 7136)

In [145]:
vect = CountVectorizer(max_df = 0.8,
                       min_df = 3,
                       stop_words = frozenset(stopwords))
term_matrix = pd.DataFrame(vect.fit_transform(X_train.cut_text).toarray(), 
                           columns = vect.get_feature_names_out())
term_matrix

/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['c螺', 'ettoday', '其他', '助詞', '動詞', '名詞', '單位量詞冠詞', '大家好我是will', '感嘆情緒', '成語俚語', '時間', '柯p', '稱呼人名', '符號', '處份', '財經m平方', '連接詞'] not in stop_words.
  warnings.warn(


,00,000,0050,01,02,03,04,05,06,07,...,鰻魚,鴻海,黃金,默默,點位,點名,點擊,鼓勵,龍頭,龍頭股
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
396,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
397,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
398,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [122]:
nb = MultinomialNB()
pipe = make_pipeline(vect, nb)
print(cross_val_score(pipe, X_train.cut_text, y_train, cv = 5, scoring='accuracy').mean())
pipe.fit(X_train.cut_text, y_train)
pipe.predict(X_test.cut_text)
y_pred = pipe.predict(X_test.cut_text)
print(metrics.accuracy_score(y_test, y_pred))
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.matthews_corrcoef(y_test, y_pred))
print(f1_score(y_test, y_pred, average='weighted'))

/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['c螺', 'ettoday', '其他', '助詞', '動詞', '名詞', '單位量詞冠詞', '大家好我是will', '感嘆情緒', '成語俚語', '時間', '柯p', '稱呼人名', '符號', '處份', '財經m平方', '連接詞'] not in stop_words.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['c螺', 'ettoday', '其他', '助詞', '動詞', '名詞', '單位量詞冠詞', '大家好我是will', '感嘆情緒', '成語俚語', '時間', '柯p', '稱呼人名', '符號', '處份', '財經m平方', '連接詞'] not in stop_words.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['c螺', 'ettoday', '其他', '助詞', '動詞', '名詞', '單位量詞冠詞', '大家好我是will', '感嘆情緒

0.6429557823942371
0.6511430990685859
[[102   6 200]
 [ 27  11 100]
 [ 59  20 656]]
0.24850003197901432
0.6048197337073877


/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['c螺', 'ettoday', '其他', '助詞', '動詞', '名詞', '單位量詞冠詞', '大家好我是will', '感嘆情緒', '成語俚語', '時間', '柯p', '稱呼人名', '符號', '處份', '財經m平方', '連接詞'] not in stop_words.
  warnings.warn(


In [126]:
from xgboost import XGBClassifier

# 建立 XGBClassifier 模型
xgboostModel = XGBClassifier(n_estimators=100, learning_rate= 0.3)

In [141]:
# 使用訓練資料訓練模型
xgb = xgboostModel
pipe = make_pipeline(vect, xgb)
pipe.fit(X_train.cut_text, y_train)

Pipeline(steps=[('countvectorizer',
                 CountVectorizer(max_df=0.8, min_df=3,
                                 stop_words=frozenset({' -', ' :// ', ' =',
                                                       ' ettoday', '!', '#',
                                                       '$', '%', '&', '(', ')',
                                                       '*', '+', '-', '--',
                                                       '.\n', '..', '...',
                                                       '....', '......', './',
                                                       '/', '//', '///', '57台',
                                                       '6666哥', ':', ':: ', ';',
                                                       '<', ...}))),
                ('xgbclassifier',
                 XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
                               colsample_bylevel=...
                               gamma=0, gpu_id=-1, grow_policy='depthwise',
                               importance_type=None, interaction_constraints='',
                               learning_rate=0.3, max_bin=256,
                               max_cat_to_onehot=4, max_delta_step=0,
                               max_depth=6, max_leaves=0, min_child_weight=1,
                               missing=nan, monotone_constraints='()',
                               n_estimators=100, n_jobs=0, num_parallel_tree=1,
                               objective='multi:softprob', predictor='auto',
                               random_state=0, reg_alpha=0, ...))])

In [140]:
# 使用訓練資料預測分類
predicted = pipe.predict(X_train.cut_text)
print('訓練集: ',pipe.score(X_train.cut_text,y_train))
print('測試集: ',pipe.score(X_test.cut_text,y_test))

訓練集:  0.7567033587355348
測試集:  0.6613039796782387
